In [3]:
import torch

import torchvision.models as models

import onnxruntime as ort

import numpy as np

Export PyTorch Model to ONNX

In [4]:
model = models.resnet18(pretrained=False)

model.load_state_dict(torch.load('resnet18_traced.pt', map_location='cpu'))

model.eval()

/opt/anaconda3/envs/masterxdl/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/envs/masterxdl/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/opt/anaconda3/envs/masterxdl/lib/python3.9/site-packages/torch/serialization.py:1413: UserWarning: 'torch.load' received a zip file that looks like a TorchScript archive dispatching to 'torch.jit.load' (call 'torch.jit.load' directly to silence this warning)
  warnings.warn(


TypeError: Expected state_dict to be dict-like, got <class 'torch.jit._script.RecursiveScriptModule'>.

We need a dummy input of the same shape as the model expects:

In [ ]:
dummy_input = torch.randn(1, 3, 32, 32)  # For CIFAR-10

Export

In [ ]:
torch.onnx.export(

    model,
    dummy_input,
    'resnet18_traced.onnx',
    input_names = ['input'],
    output_names = ['output'],
    dynamic_axes={'input' :{0,'batch_size'}, 'output' :{0,'batch_size'} },

    opset_version=11
)

print("Model exported to ONNX successfully.")


Load and Run Inference using onnxruntime

In [ ]:
# Create inference session

session = ort.InferenceSession('resnet18_traced.onnx')

In [ ]:
# Prepare input

input_data = np.random.randn(1, 3, 32, 32).astype(np.float32)

In [ ]:
# Run inference

outputs = session.run(

    None,

    {'input': input_data}
)

preds = np.argmax(outputs[0], axis=1)

print("Predicted class:", preds)